In [1]:
from qdrant_client import AsyncQdrantClient, models
import asyncio
import os
import time
from qdrant_client.models import VectorParams
from qdrant_client import AsyncQdrantClient, models
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid
from langchain_community.document_loaders import WebBaseLoader

In [2]:
# For performing asynchronous calls in a notebook
import nest_asyncio
nest_asyncio.apply()

In [3]:
os.environ["QDRANT_API_KEY"] = "vCPG1V8yO1kaGzTMWA9y5ql8nn8u-HWiHiKlTNNdE3MKrEI14efpCg"

In [4]:
# Load the webpage using its url
loader = WebBaseLoader("https://docs.ondo.finance/", verify_ssl=False)
loader.requests_kwargs = {'verify': False}

In [5]:
data_from_website = loader.load()

c:\Users\HP\Desktop\Chat with any website\.venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'docs.ondo.finance'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [6]:
data_from_website

[Document(page_content="\n\n\n\n\nWelcome to Ondo | Ondo Finance Docs\n\n\n\n\n\n\n\n\nSkip to main contentOndo Finance DocsTwitterGitHubWelcome to OndoTrust & SecurityGeneral-Access ProductsUSDY TokenUSDY BasicsComparison to StablecoinsComparison to Other Ondo ProductsHow it WorksFAQUSDY Legal DisclaimerQualified-Access ProductsCash Management 101Onboarding & KYC for Ondo I LPEligibilityOMMF Token (Coming Soon)OUSG TokenHow It WorksAsset StrategyRisk MetricsProtocolsFlux FinanceDeveloper GuidesOndo Token Tools (Bridge and Converter)Ondo Token BridgeOndo Token ConverterUSDY and mUSD ConversionSmart Contract AddressesSmart Contract AuditsOndo V1 — Vaults & LaaSLegalWelcome to OndoOn this pageWelcome to OndoAt Ondo, our mission is to make institutional-grade financial products and services available to everyone.  We believe that blockchain technology has the potential to improve both the infrastructure of — and access to — financial products and services. We also believe that the best te

In [7]:
# Chunk data from website
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200) #Initialize text splitter
chunks_of_text = text_splitter.split_documents(data_from_website)

In [8]:
chunks_of_text

[Document(page_content='Welcome to Ondo | Ondo Finance Docs', metadata={'source': 'https://docs.ondo.finance/', 'title': 'Welcome to Ondo | Ondo Finance Docs', 'description': 'At Ondo, our mission is to make institutional-grade financial products and services available to everyone.  We believe that blockchain technology has the potential to improve both the infrastructure of — and access to — financial products and services. We also believe that the best technical improvements need to be combined with the best practices from traditional finance, including investor protections, transparency in reporting, legal and regulatory compliance, intelligent and robust product structuring, working with the best service providers, and top-notch client service.', 'language': 'en'}),
 Document(page_content='Skip to main contentOndo Finance DocsTwitterGitHubWelcome to OndoTrust & SecurityGeneral-Access ProductsUSDY TokenUSDY BasicsComparison to StablecoinsComparison to Other Ondo ProductsHow it Works

In [9]:
qdrant_client = AsyncQdrantClient(
    url="https://dd489e3d-8648-47fe-88cf-156f8e9e2c90.us-east4-0.gcp.cloud.qdrant.io:6333", 
    api_key="vCPG1V8yO1kaGzTMWA9y5ql8nn8u-HWiHiKlTNNdE3MKrEI14efpCg",
)

In [10]:
name_of_collection = 'user_collection'

In [11]:
# Get the names of all existing collections and append to list
info = await qdrant_client.get_collections()
existing_collections = [] 
for each_collection in info.collections:
    existing_collections.append(each_collection.name)

In [12]:
existing_collections

['user_collection']

In [13]:
collectionExists = True if name_of_collection in existing_collections else False

In [14]:
# If collection does not exist create one using qdrant client
if collectionExists == False:
  await qdrant_client.create_collection(
  collection_name=name_of_collection,
  vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE),
  )
  print('Collection Created!')

In [15]:
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

In [16]:
# configure my embedding model from huggingface
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings_model = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

c:\Users\HP\Desktop\Chat with any website\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
vector_store = Qdrant.from_documents(
    chunks_of_text,
    embeddings_model,
    url="https://dd489e3d-8648-47fe-88cf-156f8e9e2c90.us-east4-0.gcp.cloud.qdrant.io:6333", 
    api_key="vCPG1V8yO1kaGzTMWA9y5ql8nn8u-HWiHiKlTNNdE3MKrEI14efpCg",
    collection_name=name_of_collection,
    force_recreate=True
)

# Hook up retrieval chain